# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

In [1]:
import os
import requests
import pandas as pd
from tqdm import tqdm
from sqlalchemy import create_engine

C:\Users\darmorezov\AppData\Local\Programs\Python\Python38-32\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
def send_request(url):
    """
        Функция необходима для отправки запросов и получения ответов на запросы к сайтам
        input:  url - адрес ответ с которого планируется получить
        output: response - ответ от адреса в формате json
    """
    try:
        response = requests.get(url, timeout=(10, 20) )
        if response.status_code != 200:
            raise ConnectionError('Возвращен неправильный результат')
    except Exception as error:
        print(f'При запросе произошла ошибка: {error}')
        return None
    return response.json()

In [3]:
def search_objects_ids(limit=1000, offset=1):
    """
        Функция необходима для поиска id строящихся объектов
        input:  limit - количество записей для выгрузки,
                offset - с какой записи начинать выгрузку
        output: objects_ids - id объектов строящихся зданий
    """
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset}&limit={limit}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0' 
    response = send_request(url)
    objects_ids = None
    if response:
            objects_ids = [item['objId'] for item in response['data']['list']]
    return objects_ids

In [4]:
def search_object_data(object_id):
    """
        Функция для поиска информации о строящемся объекте по его id
        input:  object_id - id объекта для которого производится поиск информации
        output: result_df - строка датафрейма, содержащая информацию о строящемся объекте
    """
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{object_id}'
    response = send_request(url)
    result_df = None
    if response:
        result_df = pd.json_normalize(response['data'])
    return result_df

In [5]:
def save_frame_to_database(dataframe, database_name, table_name):
    """
        Функция для записи данных датафрейма в базу данных
        input:  dataframe     - фрейм данных pandas,
                database_name - название базы данных
                table_name    - название таблицы базы данных
    """
    try:
        engine = create_engine('sqlite:///{0}.db'.format(database_name),
                               echo = True)
        sqlite_connection = engine.connect()
        dataframe.to_sql(table_name,
                         sqlite_connection,
                         if_exists = 'replace')
        sqlite_connection.commit()
        sqlite_connection.close()
    except Exception as error:
        print(f'Во время работы с базой данных возникла ошибка: {error}')

In [6]:
obj_ids = search_objects_ids()

In [7]:
res_df = pd.DataFrame()
for obj_id in tqdm(obj_ids):
    df = search_object_data(obj_id)
    res_df = pd.concat([res_df, df], ignore_index=True, sort=False)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:41<00:00,  6.18it/s]


In [8]:
res_df.fillna(0)
res_df = res_df.applymap(str)

In [ ]:
res_df.head(2)

In [ ]:
save_frame_to_database(res_df, 'PARSER', 'DOMRF')

In [9]:
res_df.to_pickle('result_frame.pkl')
res_df.to_excel('result_frame.xlsx')

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"

In [ ]:
def save_image(url, image_name):
    """
        Функция для сохранения изображения из сети интернет
        input:  url - адрес нахождения изображения в сети интернет
                image_name - название изображения для сохранения 
                            (в названии может быть зашит path)
    """
    with open(image_name, 'wb') as file:
        p = requests.get(url)
        file.write(p.content)

In [ ]:
def save_objects_images(photos_info_list):
    """
        Функция для сохранения фотографий строящихся зданий
        input: photos_info_list - список элементов, содержащих
                                  информацию по фотографиям к строящемуся объекту
    """
    for object_photo_info in photos_info_list:
        try:
            object_id = object_photo_info[0]['objId']
            os.mkdir(str(object_id))
            for el in object_photo_info:
                save_image(el['objRenderPhotoUrl'], '{0}//{1}.jpg'.format(object_id, el['objRenderPhotoNm']))
        except Exception as error:
            print(f'Во время работы возникла ошибка - {error}')

In [ ]:
objects_photos_info = res_df['photoRenderDTO'].tolist()

In [ ]:
save_objects_images(objects_photos_info[:10])